# Practica 3: Sensado y An&aacute;lisis Inercial 

<style>
    .imagen-ajustada {
        width: 900px;
        height: 280px;
        border-radius: 20px;
    }
</style>

<img src="Sentadillas.png" alt="Una imagen" class="imagen-ajustada">

En el notebook que se presenta a continuaci&oacute;n se tienen que procesar los datos inerciales por medio de sensores IMU, esta pr&aacute;ctica se hace con el fin de clasificar si la persona que se encuentra haciendo sentadillas lo hace de forma correcta o incorrecta, con el fin de ayudar a que se hagan de forma correcta y eviten lesiones por hacer ejercicios de forma incorrecta.
Los sensores IMU se pusieron uno en cada pantorrilla, uno en cada muslo y uno en la espalda baja del participante, siendo un total de 5 sensores. En total  fueron 10 participantes voluntarios en la toma de sus datos y as&iacute; utilizarlos en la pr&aacute;ctica, logrando as&iacute; poder entrenar nuestro modelo de forma eficiente.    

## Librer&iacute;as Importadas

## Manejo del dataset

## An&aacute;lisis Exploratorio

## Procesamiento de Datos

## Extracci&oacute;n de Caracter&iacute;sticas

## Selecci&oacute;n de Caracter&iacute;sticas

## Validaci&oacute;n de Datos

## Modelos a Evaluar

## Evaluando el mejor modelo

## Entrenando al modelo

## Implementando el modelo

## Conclusiones